In [1]:
import os

In [2]:
%pwd

'/home/priyanshu1303d/Projects/Chicken_Disease_Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'/home/priyanshu1303d/Projects/Chicken_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data : Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

2025-02-06 18:59:03.329483: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 18:59:03.447777: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 18:59:03.549520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738848543.638218    5966 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738848543.662467    5966 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 18:59:03.880175: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml , create_directories , save_json

In [29]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.keras',
            training_data='artifacts/data_ingestion/Chicken-fecal-images/',
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE)
        return eval_config
        
        

In [30]:
from urllib.parse import urlparse

In [35]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path : Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path('scores.json'),data = scores)

In [36]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evalu = Evaluation(val_config)
    evalu.evaluation()
    evalu.save_score()
except Exception as e:
    raise e

[2025-02-06 19:31:51,727: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-02-06 19:31:51,731: INFO: common : yaml file: params.yaml loaded successfully]
[2025-02-06 19:31:51,732: INFO: common : created directory at: artifacts]
Found 150 images belonging to 2 classes.


/home/priyanshu1303d/anaconda3/envs/chicken/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9200 - loss: 2.1633
[2025-02-06 19:32:12,240: INFO: common : json file saved at : scores.json]
